## **Conversational Threads**

In [2]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

### **Group traces into threads**
A Thread is a sequence of traces representing a single conversation. Each response is represented as its own trace, but these traces are linked together by being part of the same thread.

To associate traces together, you need to pass in a special metadata key where the value is the unique identifier for that thread.

The key value is the unique identifier for that conversation. The key name should be one of:

- session_id
- thread_id
- conversation_id.

The value should be a UUID.

In [ ]:
import uuid
thread_id = uuid.uuid4()

from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type = "chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type = "chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the latest question in the conversation. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    """
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable(run_type = "llm")
def call_openai(
    messages: List[dict], model: str = "gpt-4o-mini", temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature,
    )

@traceable(run_type = "chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

In [4]:
question = "How do I add metadata to a Trace?"
ai_answer = langsmith_rag(question, langsmith_extra = {"metadata": {"thread_id": thread_id}})
print(ai_answer)

To add metadata to a trace in LangSmith, you can follow the instructions provided in the "Learn how to add metadata to your traces" section. This typically involves attaching key-value pairs to the runs you want to annotate. For detailed steps, refer to the relevant documentation on adding metadata.


In [5]:
question = "How can I add tags to a Trace?"
ai_answer = langsmith_rag(question, langsmith_extra = {"metadata": {"thread_id": thread_id}})
print(ai_answer)

You can add tags to a trace by annotating it either inline or by sending it to an annotation queue within the LangSmith application. Tags are associated with your workspace and can provide additional context about the trace, such as the environment or user details. For more information, refer to the Concepts page in the documentation.
